## build_device_catalog.ipynb

In [15]:
import pandas as pd
import json
from pathlib import Path

raw_path = Path("raw_data/smart_home_energy_consumption_large.csv")
output_path = Path("data/devices_catalog.json")


In [17]:
df = pd.read_csv(raw_path)
print(df.shape)
df.head(50)


(100000, 8)


,Home ID,Appliance Type,Energy Consumption (kWh),Time,Date,Outdoor Temperature (°C),Season,Household Size
0,94,Fridge,0.20,21:12,2023-12-02,-1.0,Fall,2
1,435,Oven,0.23,20:11,2023-08-06,31.1,Summer,5
2,466,Dishwasher,0.32,06:39,2023-11-21,21.3,Fall,3
3,496,Heater,3.92,21:56,2023-01-21,-4.2,Winter,1
4,137,Microwave,0.44,04:31,2023-08-26,34.5,Summer,5
5,68,Air Conditioning,4.68,11:36,2023-05-06,35.2,Spring,1
6,237,Computer,0.25,12:05,2023-06-06,6.8,Spring,3
7,329,Air Conditioning,3.50,03:34,2023-12-12,-1.0,Fall,4
8,336,Dishwasher,0.89,15:22,2023-08-16,7.9,Summer,1
9,310,Microwave,1.34,16:32,2023-09-19,9.3,Summer,5


In [6]:
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   home_id                   100000 non-null  int64  
 1   appliance_type            100000 non-null  object 
 2   energy_consumption_(kwh)  100000 non-null  float64
 3   time                      100000 non-null  object 
 4   date                      100000 non-null  object 
 5   outdoor_temperature_(°c)  100000 non-null  float64
 6   season                    100000 non-null  object 
 7   household_size            100000 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 6.1+ MB


In [8]:
# Drop any rows missing device or kWh values
df = df.dropna(subset=["appliance_type", "energy_consumption_(kwh)"])

# Make sure numeric columns are correct types
df["energy_consumption_(kwh)"] = pd.to_numeric(df["energy_consumption_(kwh)"], errors="coerce")

# Remove any remaining NaN
df = df.dropna(subset=["energy_consumption_(kwh)"])


In [12]:
summary = (
    df.groupby("appliance_type")["energy_consumption_(kwh)"]
      .mean()
      .reset_index()
      .sort_values("energy_consumption_(kwh)", ascending=False)
      .rename(columns={
          "appliance_type": "device",
          "energy_consumption_(kwh)": "avg_kwh"
      })
)

summary.head(100)


,device,avg_kwh
0,Air Conditioning,3.499857
4,Heater,3.486802
2,Dishwasher,1.103369
7,Oven,1.103080
9,Washing Machine,1.101412
6,Microwave,1.100961
8,TV,1.097221
1,Computer,1.095498
5,Lights,1.087356
3,Fridge,0.298255


In [13]:
catalog = {}
for _, row in summary.iterrows():
    name = row["device"].strip().title()  # clean capitalization
    base_kwh = round(float(row["avg_kwh"]), 3)
    catalog[name] = {
        "base_kWh": base_kwh,
        "permissions": []
    }

In [16]:
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(catalog, f, indent=2, ensure_ascii=False)

print(f"✅ Exported {len(catalog)} devices to {output_path}")

✅ Exported 10 devices to data\devices_catalog.json
